In [3]:
import urllib.request
import zipfile

# Download the zip file
url = "https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip"
urllib.request.urlretrieve(url, "bike_sharing_dataset.zip")

# Unzip the file
with zipfile.ZipFile("bike_sharing_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("./extracted_data")

In [4]:
import pandas as pd
# Load the CSV file
df = pd.read_csv('./extracted_data/hour.csv') 
df

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


In [5]:
df['day_night'] = df['hr'].apply(lambda x: 'day' if 6 <= x <= 18 else 'night')
# Creating interaction features
df['temp_hum'] = df['temp'] * df['hum']
df['temp_windspeed'] = df['temp'] * df['windspeed']

df.drop(['instant', 'casual', 'registered'], axis=1, inplace=True)
df['dteday'] = pd.to_datetime(df.dteday)
df['season'] = df.season.astype('category')
df['holiday'] = df.holiday.astype('category')
df['weekday'] = df.weekday.astype('category')
df['weathersit'] = df.weathersit.astype('category')
df['workingday'] = df.workingday.astype('category')
df['mnth'] = df.mnth.astype('category')
df['yr'] = df.yr.astype('category')
df['hr'] = df.hr.astype('category')
df.drop(columns=['dteday'], inplace=True)

# Separating features and target variable
X = df.drop(columns=['cnt']) # Features
y = df['cnt'] # Target

df

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt,day_night,temp_hum,temp_windspeed
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,16,night,0.1944,0.000000
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,40,night,0.1760,0.000000
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,32,night,0.1760,0.000000
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,13,night,0.1800,0.000000
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,1,night,0.1800,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,119,night,0.1560,0.042692
17375,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,89,night,0.1560,0.042692
17376,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,90,night,0.1560,0.042692
17377,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,61,night,0.1456,0.034918


In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler#, TargetEncoder
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder

### Replace the OneHotEncoder with TargetEncoder for categorical variables.

In [7]:
# Numerical features
numerical_features = ['temp', 'hum', 'windspeed']
numerical_pipeline = Pipeline([
('imputer', SimpleImputer(strategy='mean')), # Impute missing values with mean
('scaler', MinMaxScaler()) # Normalize using MinMaxScaler
])
# Transforming above
X[numerical_features] = numerical_pipeline.fit_transform(X[numerical_features])
# Categorical features
categorical_features = ['season', 'weathersit', 'day_night']
categorical_pipeline = Pipeline([
('imputer', SimpleImputer(strategy='most_frequent')),
('target', TargetEncoder())
])
# Transforming above
X_encoded = categorical_pipeline.fit_transform(X[categorical_features],y)
# Converting it to a dataframe

X_encoded = pd.DataFrame(X_encoded,
columns=categorical_pipeline.named_steps['target'].get_feature_names_out(categorical_features))
# Encoded categorical features + Numerical features
X = pd.concat([X.drop(columns=categorical_features), X_encoded], axis=1)

X.columns = X.columns.astype(str)

c:\Users\DELL\anaconda3\envs\qiskit\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\DELL\anaconda3\envs\qiskit\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\DELL\anaconda3\envs\qiskit\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ff

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=42)


model = RandomForestRegressor( n_estimators=100,random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [9]:
# Predictions
from sklearn.metrics import *

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 1778.451213697555
R-squared: 0.9438361562778752


### Linear Regression

In [10]:
from sklearn.linear_model import LinearRegression

sklearn_model = LinearRegression()

sklearn_model.fit(X_train, y_train)

y_pred_sklearn = sklearn_model.predict(X_test)

mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)
r2_sklearn = r2_score(y_test, y_pred_sklearn)

print(f"Sklearn Linear Regression - MSE: {mse_sklearn}, R²: {r2_sklearn}")

Sklearn Linear Regression - MSE: 14974.13386064109, R²: 0.5271138687719743


In [11]:
import numpy as np
X_train_aug = np.concatenate([np.ones((X_train.shape[0], 1)), X_train], axis=1)
X_test_aug = np.concatenate([np.ones((X_test.shape[0], 1)), X_test], axis=1)

optimal_theta = np.linalg.inv(X_train_aug.T @ X_train_aug) @ X_train_aug.T @ y_train

predictions = X_test_aug @ optimal_theta

mse_manual = mean_squared_error(y_test, predictions)
r2_manual = r2_score(y_test, predictions)

print(f"Custom Linear Regression - MSE: {mse_manual}, R²: {r2_manual}")


Custom Linear Regression - MSE: 14974.133860640986, R²: 0.5271138687719774


In [12]:
from sklearn import set_config
final_pipeline = Pipeline([
('num_preprocess', numerical_pipeline),
('cat_preprocess', categorical_pipeline),
('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

set_config(display='diagram')# To display
final_pipeline

Pipeline(steps=[('num_preprocess',
                 Pipeline(steps=[('imputer', SimpleImputer()),
                                 ('scaler', MinMaxScaler())])),
                ('cat_preprocess',
                 Pipeline(steps=[('imputer',
                                  SimpleImputer(strategy='most_frequent')),
                                 ('target', TargetEncoder(cols=[0, 1, 2]))])),
                ('model', RandomForestRegressor(random_state=42))])